# Migrating COM API calls (usually used in Excel & VBA) to Python

### What are the COM APIs

The COM APIs are older APIs that were initially delivered with 3000 Xtra Terminals and then Eikon. They provided access to a broad set of content and capabilities in Eikon.

The COM API covers a large range of functionalities, as shown in [this article](https://developers.refinitiv.com/en/article-catalog/article/event-driven-financial-calculation-with-eikon-excel-visual-basic). Below we will show you how to migrate the following COM Component and API calls/functions: 



* AdfinXRtLib (rtx.dll)
    * AdxRtSourceList 
    * AdxRtList (including 'RtGet Real Time')
    * AdxRtChain
    * AdxRtHistory (both inter- and intra-day) - Only available in Excel
    * Streaming News


* dex2.dll 
    * Rdata List
    * DEX2 (including Symbology Conversion, Time Series, Fundamental and Reference data) 


* RSearch.dll
    * RSearch 


* RHistoryAPI.dll
    * RHistory


* RtContribute.dll
    * RtContribute 


----The APIs below do not currently have replacements in Workspace---------------


* Adxfo.dll
    * AdfinX Analytics 2.0 Module


* Adxfoo.dll
    * AdfinX Analytics 3.0 Module




### Reasons you might need or want to migrate away from COM APIs:

### Need to operate in 64-bit environment
   
The COM APIs were designed to be used on 32-bit environments. Many teams are stil migrating to 64-bit environments where these will not work correctly and are not supported environments.

### You want access to new datasets and improved capabilties offered on Refinitiv Data Platform

The COM APIs are feature-complete and no new development work is being done on them. This means that new datasets    and capabilities will not be available to users of these APIs.

### You want to upgrade from Eikon to Workspace

The COM APIs will not be available in Workspace - so to take advantage of the best desktop experience you would need to migrate to our more modern web-based APIs - which are much better and work with modern scripting environments such as Python - one of the most popular language ecosystems used in finance. 


All the Python code below works in [Codebook](cpurl://apps.cp./Apps/code-book/) too! You can try it out as outlined in this [Tutorial Video](https://developers.refinitiv.com/en/video-catalog/from-developer-articles-to-codebook). We will use the Refinitiv Data Library (RD Library). To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like
- [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
- [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
- [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)

and much more.

## Some Imports to start with

If you are new to [Python](https://www.python.org/), don't hesitate to install it on your machine and try it out yourslef as outlined in this 3rd party [tutorial](https://pythonprogramming.net/introduction-learn-python-3-tutorials/). Elsehow, you can simply use [Codebook](cpurl://apps.cp./Apps/code-book/) as outlined in this [Tutorial Video](https://developers.refinitiv.com/en/video-catalog/from-developer-articles-to-codebook).

Python works with libraries that one can import to use functionalities that are not natively supported but the coding language. In the cell below, we import the Python library `os`, then we use it to point to the path where our Refinitiv Credentials are; we need this file in order to auithenticate ourselves to the Refinitiv data services and collect data. You can find out more on this [here](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), and a copy of the Configuration file [here](https://github.com/Refinitiv-API-Samples/Example.DataLibrary.Python/blob/main/Configuration/refinitiv-data.config.json).

In [3]:
import refinitiv.data as rd  # pip install httpx==0.21.3 # !pip install refinitiv.data --upgrade
import pandas as pd
import numpy as np
import os

In [4]:
# 1st we need to point to the configuration file on our machine:
#rd.open_session(config_name="C:\\Example.DataLibrary.Python-main\\Example.DataLibrary.Python-main\\Configuration\\refinitiv-data.config.json")
# Now we can choose the session type we would like. You can use `rd.open_session("desktop.workspace")` if running this code outside CodeBook. You can also try "platform.rdp" or even `rd.open_session("")`.
rd.open_session("desktop.workspace")

<refinitiv.data.session.Definition object at 0x7f94605ed390 {name='workspace'}>

## AdfinX RealTime AdxRtSourceList
### What does this do: 
The functionality in Office was used to get the real-time field list for the selected Source Name, such as an instrument (e.g.: VOD.L) (using AdxRtSourceList Class of AdfinXRtLib, the AdfinX Real Time 6.0 Library, rtx.dll). It was best exemplified in the [Tutorial 1 - Real-time Source List, Fields - AdxRtSourceList](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-1-real-time-source-list-fields-adx-rt-source-list)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/adxrtsourcelist.zip):

<img src="imgs/AdxRtSourceList.PNG">

### New Method
This is a functionality for which there is now an interface available called the [Data Item Browswer (DIB) that you can  find on workspace](https://emea1-apps.platform.refinitiv.com/web/Apps/DataItemBrowser/), and for which there is a [video tutorial](https://developers.refinitiv.com/en/video-catalog/data-item-browser).

## AdfinX RealTime AdxRtList
### What does this do?

Returns real-time data for multiple (or single) instrument and fields. Data is returned initially as ONIMAGE - which is a snapshot of data for the requested fields, followed by a series of ONUPDATE messages - which are received whenever a data item changes or gets updated. You can also request ONTIME returns which would give an ONIMAGE snapshot at periodic intervals (say every hour for example). All the following functions rely on AdxRtList API calls - RtGet(), RData() and TR() for real-time data. Typically we would need to write callback handlers to deal with the returns from the API.

### RtGet Real-Time

When using the old COM API to get `RtGet Real Time` data, one may be greeted with an Excel sheet that looks like the below, as per [Tutorial 2 - Real-time Data Retrieval - AdxRtList](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-2-real-time-data-retrieval-adx-rt-list)'s [Excel workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/adxrtlist.zip):

<img src="imgs/1. RtGet Real Time Excel Front End Eg.PNG">

The functionalities shown here are easily recreated using the RD Library: 

1. Real Time FX rates snapshot - 'Real-Time ONIMAGE' 
2. 'Real Time Update', which updates real time, as soon as an update for the instrument and field is received 
3. A periodic ONTIME snapshot

#### Real-Time ONIMAGE

When collecting data ONIMAGE, we are collecting a current snapshot of the data we're after. This could not be simpler in RD in Python!

##### VBA

In VBA, you'd create your function `cmdGetRealTimeONIMAGE_Click`:

>         Private Sub cmdGetRealTimeONIMAGE_Click()
>         Dim strRICs As String  ' Can have one or more items
>         Dim varFIDs As Variant ' Field can be numeric as well as a string, e.g. BID is field 22
>         
>         ActiveCell.Select
>         
>         If Not myRtGet Is Nothing Then Set myRtGet = Nothing
>         Set myRtGet = CreateAdxRtList()
>         
>         With myRtGet
>             .ErrorMode = DialogBox
>             .Source = [Source].Value
>             strRICs = [RIC].Value
>             varFIDs = [FID].Value
> 
>             .RegisterItems strRICs, varFIDs
>     '       'Different methods shown below.
>     '        strRICs = "EUR="
>     '        varFIDs = "BID"
>     '        .RegisterItems strRICs, varFIDs
>     '        .RegisterItems "EUR=,GBP=,JPY=", "BID,ASK"
> 
>             .StartUpdates RT_MODE_IMAGE ' 4
>             '.StartUpdates RT_MODE_ONUPDATE ' 3
>             '.StartUpdates RT_MODE_NOT_SET ' 5
>             '.StartUpdates RT_MODE_ONTIME ' 2
>             '.StartUpdates RT_MODE_ONTIME_IF_UPDATED ' 1
>         End With ' For the With myRtGet
>     End Sub

This would allow, in this example, for the 'Get Real Time ONIMAGE' buttons to work. For updates, the below could be used:

>     ' Returns the initial image for the instrument. NOTE - .StartUpdates RT_MODE_IMAGE
>     Private Sub myRtGet_OnImage(ByVal DataStatus As AdfinXRtLib.RT_DataStatus)
>         Dim arrRICs As Variant, arrFields As Variant
>         Dim lngRICFidVal As Single
>         Dim a As Integer
> 
>         If DataStatus = RT_DS_FULL Then
>             With myRtGet
>                 ' Array of the list of instruments - only one in this case.
>                 arrRICs = .ListItems(RT_IRV_ALL, RT_ICV_USERTAG)
>                 ' Array of the list of Fields for the ath item in the arrRics (base 0)
>                 a = 0
>                 arrFields = .ListFields(arrRICs(a, 0), RT_FRV_ALL, RT_FCV_VALUE)
>             End With
> 
>             ' And a specific value for a specific instrument, specific field.
>             'lngRICFidVal = myRtGet.Value("EUR=", "BID")
>             lngRICFidVal = myRtGet.Value([RIC].Value, [FID].Value)
>             [F7].Value = lngRICFidVal
>         End If
>     End Sub

##### Python

In python, things could not be easier as most of the equivalent code about is abstracted to the library and you just use a one-line function! You can go ahead and try it all out in [Codebook](cpurl://apps.cp./Apps/code-book/):

In [4]:
rd.get_data(
    ['GBP=', 'EUR=', 'JPY='],
    ['BID', 'ASK'])

,Instrument,BID,ASK
0,GBP=,1.1576,1.158
1,EUR=,0.9996,1.0
2,JPY=,142.6,142.63


You can easily assign this info to an object too.

In [5]:
realTimeImage = rd.get_data(
    ['GBP=', 'EUR=', 'JPY='],
    ['BID', 'ASK'])
realTimeImage

,Instrument,BID,ASK
0,GBP=,1.1576,1.158
1,EUR=,0.9997,0.9998
2,JPY=,142.62,142.63


##### Real-Time Fields Available With The COM API

In addition to the 'AdxRtSourceList'shown above, you you could lookup fields available in the Quote app for any instrument in question:

<img src="imgs/COM Real Time Fields1.PNG">

<img src="imgs/COM Real Time Fields2.PNG">

Note that available fields for each instrument type difffers on the type.

##### Real-Time Fields Available In Python

You can get a collection of all the Real-Time fields available via:

In [6]:
RTCurrDf = rd.get_data(['GBP='])
print(list(RTCurrDf.columns))

['Instrument', 'PROD_PERM', 'RDNDISPLAY', 'DSPLY_NAME', 'TIMACT', 'NETCHNG_1', 'HIGH_1', 'LOW_1', 'CURRENCY', 'ACTIV_DATE', 'OPEN_PRC', 'HST_CLOSE', 'BID', 'BID_1', 'BID_2', 'ASK', 'ASK_1', 'ASK_2', 'ACVOL_1', 'TRD_UNITS', 'PCTCHNG', 'OPEN_BID', 'OPEN_ASK', 'CLOSE_BID', 'CLOSE_ASK', 'NUM_MOVES', 'OFFCL_CODE', 'HSTCLSDATE', 'YRHIGH', 'YRLOW', 'BCKGRNDPAG', 'BID_NET_CH', 'BID_TICK_1', 'MID_PRICE', 'MID_NET_CH', 'MID_CLOSE', 'HIGHTP_1', 'LOWTP_1', 'BID_HIGH_1', 'BID_LOW_1', 'YRBIDHIGH', 'YRBIDLOW', 'HST_CLSBID', 'HSTCLBDDAT', 'NUM_BIDS', 'RECORDTYPE', 'ACT_TP_1', 'ACT_TP_2', 'ACT_TP_3', 'SEC_ACT_1', 'SEC_ACT_2', 'SEC_ACT_3', 'SC_ACT_TP1', 'SC_ACT_TP2', 'SC_ACT_TP3', 'OPEN_TIME', 'HIGH_TIME', 'LOW_TIME', 'YRHIGHDAT', 'YRLOWDAT', 'IRGPRC', 'TIMCOR', 'PRIMACT_1', 'PRIMACT_2', 'PRIMACT_3', 'BASE_CCY', 'BCAST_REF', 'CROSS_SC', 'DLG_CODE1', 'DLG_CODE2', 'DLG_CODE3', 'CTBTR_1', 'CTBTR_2', 'CTBTR_3', 'CTB_LOC1', 'CTB_LOC2', 'CTB_LOC3', 'CTB_PAGE1', 'CTB_PAGE2', 'CTB_PAGE3', 'VALUE_DT1', 'VALUE_DT

Note that available fields for each instrument type difffers on the type, e.g.:

In [7]:
ATMIVDf = rd.get_data(['AAPLATMIV.U'])

In [8]:
print(list(ATMIVDf.columns))

['Instrument', 'PROD_PERM', 'RDNDISPLAY', 'DSPLY_NAME', 'RDN_EXCHID', 'TIMACT', 'CURRENCY', 'TRADE_DATE', 'TRDTIM_1', 'TRD_UNITS', 'RECORDTYPE', 'SALTIM', 'BEST_BID1', 'BEST_BID2', 'BEST_BID3', 'BEST_ASK1', 'BEST_ASK2', 'BEST_ASK3', 'BCAST_REF', 'BEST_BSIZ1', 'BEST_BSIZ2', 'BEST_BSIZ3', 'BEST_ASIZ1', 'BEST_ASIZ2', 'BEST_ASIZ3', 'LONGLINK1', 'GV1_TEXT', 'GV2_TEXT', 'GV4_TEXT', 'PREF_DISP', 'PREF_LINK', 'DSPLY_NMLL', 'DSO_ID', 'RDN_EXCHD2', 'PREV_DISP', 'UNDERLYING', 'TIMACT_MS', 'OPTIONS', '30D_A_IM_C', '30D_A_IM_P', '60D_A_IM_C', '60D_A_IM_P', '90D_A_IM_C', '90D_A_IM_P', 'CONTEXT_ID', 'CF_DATE', 'CF_EXCHNG', 'CF_SOURCE', 'CF_TIME', 'CF_NAME', 'DDS_DSO_ID', 'CF_CURR', 'SPS_SP_RIC']


#### Real-Time ONUPDATE

##### VBA

<img src="imgs/1. RtGet Real Time Excel Front End Eg.PNG">

The 'Real Time ONUPDATE' buttons in the example pictured above was coded with VBA code for `cmdGetRealTimeONUPDATE_Click`:

>     Private Sub cmdGetRealTimeONUPDATE_Click()
>         Dim strRICs As Variant, varFIDs As Variant
>     
>          ActiveCell.Select
>         
>         Set myRtGet2 = CreateAdxRtList
>     
>         With myRtGet2
>             .ErrorMode = DialogBox
>             .Source = [Source].Value
>             strRICs = [RIC].Value
>             varFIDs = [FID].Value
>     
>             .RegisterItems strRICs, varFIDs
>             .StartUpdates RT_MODE_ONUPDATE
>         End With ' For the With myRtGet2
>     End Sub
>     
>     
>     ' Returns the data for updates - NOTE .StartUpdates RT_MODE_ONUPDATE.
>     Private Sub myRtGet2_OnUpdate(ByVal a_itemName As String, ByVal a_userTag As Variant, ByVal a_itemStatus As AdfinXRtLib.RT_ItemStatus)
>         Dim arrFields As Variant
>         Dim lngRICFidVal As Long
>     
>         If a_itemStatus = RT_ITEM_OK Then
>             arrFields = myRtGet2.ListFields(a_itemName, RT_FRV_ALL, RT_FCV_VALUE)
>     
>             ' And a specific value for a specific instrument, specific field.
>             'If a_itemName = "EUR=" Then lngRICFidVal = myRtGet2.Value("EUR=", "BID"): [F12].Value = arrFields(0, 1)
>             If a_itemName = [RIC].Value Then lngRICFidVal = myRtGet2.Value([RIC].Value, [FID].Value): [F12].Value = arrFields(0, 1)
>         End If
>     End Sub

Then, on VBA, you'd have to have a buttons to stop the stream with `cmdSwitchRealTimeOFF_Click`, which, in python, is `stream.close()`:

##### Python

Some Python Library Imports to start with

In [31]:
import datetime  # `datetime` allows us to manipulate time as we would data-points.
from IPython.display import display, clear_output  # `IPython` here will allow us to plot grahs and the likes.

For us to start using pricing streams with events, we need to define a callback to receive data events:

In [32]:
def display_data(data, instrument, stream):
    clear_output(wait=True)
    current_time = datetime.datetime.now().time()
    print(current_time, "- Data received for", instrument)
    display(data)

Open the stream and register the callback

In [11]:
stream = rd.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID', 'ASK'],
    on_data=display_data
)

16:59:49.372294 - Data received for JPY=


,BID,ASK
JPY=,142.62,142.63


Close the stream

In [12]:
stream.close()

<OpenState.Closed: 'Closed'>

More related RD capabilities can be found [here](https://github.com/Refinitiv-API-Samples/Example.DataLibrary.Python/blob/main/Examples/1-Access/EX-1.01.03-PricingStream.ipynb).

#### Real-Time ONTIME

With Real-Time ONTIME, you can get snapshot updates at a specific frequency:

<img src="imgs/1. RtGet Real Time Excel Front End Eg.PNG">

We can use a Python loop with `sleep` to recreate that simply:

The cell below gets an update for instruments 'GBP=', 'EUR=' and 'JPY=' and fields 'BID' and 'ASK' every 5 seconds:

In [13]:
import time  # This module provides various time-related functions. For related functionality, see also the datetime and calendar modules. We will use it to force our code to wait some time.

for i in range(3):  # This for loop will stop after 3 iterations, but you could let it run continuously by replacing it with a while loop: `while True:`
    time.sleep(5)
    display(rd.get_data(
        ['GBP=', 'EUR=', 'JPY='],
        ['BID', 'ASK']))

,Instrument,BID,ASK
0,GBP=,1.1578,1.1582
1,EUR=,0.9996,1.0
2,JPY=,142.62,142.65


,Instrument,BID,ASK
0,GBP=,1.1577,1.1581
1,EUR=,0.9996,0.9999
2,JPY=,142.64,142.67


,Instrument,BID,ASK
0,GBP=,1.1578,1.1582
1,EUR=,0.9997,0.9998
2,JPY=,142.62,142.65


##### Real Time ONTIME Note on chains used in `ek.get_data`:

It has to be pointed out that when getting this kind of data for each constituent of a chain (e.g.: `0#.FSTE` for the FTSE Equity index, or the commodities chain `0#LCO:` for  ICE Brent Crude Oil Chain Future Contract) on the COM API in Excel:

<img src="imgs/2. RData Excel Front End Eg.PNG">

you would not need the starting `0#`, but this is needed for our Python API:

# for commodity chains we need to use EDAPI, what does the symbology service? Jason to speak to Olivier tmr

In [14]:
import refinitiv.dataplatform.eikon as ek
ek.set_app_key('5e69a8b3903a4c50ab44d4cc565db992d85b81ef')
## Outside CodeBook:
#ek_key = open("eikon.txt", "r") # The key is placed in a text file so that it may be used in this code without showing it itself.
#ek.set_app_key(str(ek_key.read())) # `'DEFAULT_CODE_BOOK_APP_KEY'` in CodeBook
#ek_key.close() # It is best to close the files we opened in order to make sure that we don't stop any other services/programs from accessing them if they need to.

In [15]:
df, err = ek.get_data(
    instruments=['0#LCO:'], # other e.g.: PSKc1  # in Office COM API, you do not need the 0#, but it is requiered on our Python API calls
    fields=['CF_NAME', 'CF_TIME', 'HST_VOL'])
df

,Instrument,CF_NAME,CF_TIME,HST_VOL
0,LCOTOT,BRENT CRUDE VOLS,11:06:37,1046542
1,LCOZ2,BRENT CRUDE DEC2,11:06:23,272350
2,LCOF3,BRENT CRUDE JAN3,11:06:34,260248
3,LCOG3,BRENT CRUDE FEB3,11:06:23,145831
4,LCOH3,BRENT CRUDE MAR3,11:06:04,97380
...,...,...,...,...
72,LCOX8,BRENT CRUDE NOV8,00:00:00,<NA>
73,LCOZ8,BRENT CRUDE DEC8,11:06:38,<NA>
74,LCOF9,BRENT CRUDE JAN9,00:00:00,<NA>
75,LCOG9,BRENT CRUDE FEB9,00:00:00,<NA>


### NEW FUNCTIONALITY: Record ticks

With the RD library we now have the ability to record a pricing stream. Here's how:

Create and open a Pricing stream

In [16]:
stream = rd.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID']
)

Start recording

In [17]:
stream.recorder.record(frequency='tick')

... Wait for a little while (5 seconds) ...

In [18]:
time.sleep(5)

Stop recording and display the recorded history

In [19]:
stream.recorder.stop()
tick_history = stream.recorder.get_history()
display(tick_history)

,JPY=,EUR=,GBP=
,BID,BID,BID
Timestamp,,,
2022-09-14 17:00:19.081294,<NA>,0.9998,<NA>
2022-09-14 17:00:19.262295,<NA>,<NA>,1.1577
2022-09-14 17:00:19.349297,142.62,<NA>,<NA>
2022-09-14 17:00:19.496294,<NA>,0.9996,<NA>
2022-09-14 17:00:19.530294,<NA>,<NA>,1.1579
2022-09-14 17:00:20.510294,<NA>,0.9996,<NA>
2022-09-14 17:00:20.511295,142.62,<NA>,<NA>
2022-09-14 17:00:20.829293,<NA>,<NA>,1.1579


Resample the tick history to 5 seconds bars

In [20]:
tick_history.ohlc("5s")

JPY=                            EUR=                  \
                        BID                             BID                   
                       open    high     low   close    open    high     low   
Timestamp                                                                     
2022-09-14 17:00:15  142.62  142.62  142.62  142.62  0.9998  0.9998  0.9996   
2022-09-14 17:00:20  142.62  142.63  142.61  142.62  0.9996  0.9998  0.9996   

                               GBP=                          
                                BID                          
                      close    open    high     low   close  
Timestamp                                                    
2022-09-14 17:00:15  0.9996  1.1577  1.1579  1.1577  1.1579  
2022-09-14 17:00:20  0.9996  1.1579  1.1579  1.1579  1.1579

Close the stream

In [21]:
stream.close()

<OpenState.Closed: 'Closed'>

As you can see, we get the data sought after. If you are after several fields for any one instrument (e.g.: an FX Pair), it couldn't be any simpler:

### Rdata List Realtime

When using the old COM API to get `Rdata List` data, one may be greeted with an Excel sheet that looks like this:

<img src="imgs/2. RData Excel Front End Eg.PNG">

#### VBA

In VBA, this was done with a function akin to `.StartUpdates RT_MODE_ONUPDATE` & `myRTList = CreateAdxRtList()`, e.g.:


>     With myRTList
>         .ErrorMode = EXCEPTION
>         ' N.B.! Source name may need to be changed if not named as below!
>         .Source = "IDN" '_SELECTFEED"
>         ' Register the items and fields
>         .RegisterItems ItemArray, FieldArray
> 
>         ' Set the user tag on each item. This helps indexing the results
>         ' table for displaying the data in the callback
>         For m = LBound(ItemArray) To UBound(ItemArray)
>             .UserTag(ItemArray(m), "*") = m
>             For n = LBound(FieldArray) To UBound(FieldArray)
>                 .UserTag(ItemArray(m), FieldArray(n)) = n
>             Next n
>         Next m
> 
>         .Mode = "TIMEOUT:5"
>         ' If timed basis desired, then FRQ setting and RT_MODE_ONTIME or RT_MODE_ONTIME_IF_UPDATED required,
>         ' which will trigger the OnUpdate event, shown below.
>         '.Mode = "FRQ:2S"
>         ' And, finally, request the data!
>         Select Case Range("dcUpdateType").Value
>             Case "RT_MODE_IMAGE"
>                 .StartUpdates RT_MODE_IMAGE
>             Case "RT_MODE_ONUPDATE"
>                 .StartUpdates RT_MODE_ONUPDATE
>         End Select
> 
>         '.StartUpdates RT_MODE_ONUPDATE
>         '.StartUpdates RT_MODE_IMAGE
>         'Other modes shown below; different events will be fired.
>         '.StartUpdates RT_MODE_ONTIME, RT_MODE_ONTIME_IF_UPDATED, RT_MODE_ONTIME,
>         ' RT_MODE_ONUPDATE, RT_MODE_IMAGE , RT_MODE_NOT_SET
>     End With

To stop this update, you would have to create some VBA code to (e.g.: `Sub cmdStop_Click()`), but that is simpler in Python with `stream.close()`:

However - many developers also used the RData worksheet function object directly in VBA. 

#### Python

Here we have a data-frame of instruments and fields updating live every `x` seconds, let's say (for the sake of the use-case example) every 3 seconds. This is simple to recreate in Python:

In [33]:
stream = rd.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY=', '.DIB', '.FTSE', '.NDX', 'TRI.TO', 'EURGBP=R'],
    fields=['CF_TIME', 'CF_LAST', 'BID', 'ASK', 'TRDTIM_1'])

In [34]:
stream.open()

[StreamingPrices id=3 universe=['GBP=', 'EUR=', 'JPY=', '.DIB', '.FTSE', '.NDX', 'TRI.TO', 'EURGBP=R']] can’t open, state=StreamState.Opened


<OpenState.Opened: 'Opened'>

In [35]:
import time# # This cell's code is usually commented out so that the kernel doesn't get stuck in the while loop.
while True:
    time.sleep(3.0)
    clear_output(wait=True)
    df = stream.get_snapshot(
        universe = ['GBP=', 'EUR=', 'JPY=', '.DIB', '.FTSE', '.NDX', 'TRI.TO', 'EURGBP=R'], 
        fields = ['CF_TIME', 'CF_LAST', 'BID', 'ASK', 'TRDTIM_1'])
    display(df)

,Instrument,CF_TIME,CF_LAST,BID,ASK,TRDTIM_1
0,GBP=,11:50:26,1.1233,1.1233,1.1235,<NA>
1,EUR=,11:50:26,0.9805,0.9805,0.9809,<NA>
2,JPY=,11:50:26,149.79,149.79,149.8,<NA>
3,.DIB,<NA>,<NA>,<NA>,<NA>,<NA>
4,.FTSE,11:50:00,6928.37,<NA>,<NA>,11:50:00
5,.NDX,21:15:59,11103.377,<NA>,<NA>,<NA>
6,TRI.TO,20:15:00,140.29,134.75,142.5,<NA>
7,EURGBP=R,11:50:26,0.8727,0.8727,0.8732,<NA>


KeyboardInterrupt: 

Close the stream

In [36]:
stream.close()

<OpenState.Closed: 'Closed'>

#### Create a Streaming Price and register event callbacks using RDP

You can build upon the example above, using the [RDP](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis) Example notebook present in [Codebook](cpurl://apps.cp./Apps/code-book/) that demonstrates how to use a StreamingPrice with events to update a Pandas DataFrame with real-time streaming data. Using a StreamingPrices object that way allows your application to have at its own in memory representation (a Pandas DataFrame in this example) that is kept updated with the latest streaming values received from Eikon or Refinitiv Workspace. Here we're putting ourselves in the shoes of a Foreign eXchange (FX) trader looking at Emerging Market (EM) currency exchange rates; e.g: the Nigerian Nairas (NGN) and Indonesian Rupiah (IDR).

You can find the code for this on GitHub [here](https://gist.github.com/johnukfr/25935ae4a0928c400e706dee7eb6a79a).

### RData Function

#### What does this do?
RData is a flexible excel worksheet function allowing access to realtime and fundamental & reference data content. It can also be used programatcally in VBA and the results then dumped to an excel range for example. 

#### VBA

For VBA related to Fundamental data, please see the DEX2 Fundamental and Reference section below.

#### Python

We have separated getting current fundamental snapshots - using a rd.get_data function. For historical fundamental timeseries histories we have created a rd.content.fundamental_and_reference.Definition.


Such data can be found in Python with [RD](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) (and `rd.get_data` or `rd.content.fundamental_and_reference.Definition`) or [EDAPI](https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api).

##### Snapshot requests

In [25]:
df1 = rd.get_data(
    ['BARC.L', 'TRI.N'],  # PSKc1, '92857WAZ3=1M'
    fields=[
        'TR.IR.RatingSourceDescription', 'TR.IR.RatingSourceType', 'TR.IR.Rating',
        'TR.TRBCEconomicSector', 'TR.TRBCEconSectorCode', 'TR.TRBCBusinessSector',
        'TR.TRBCBusinessSectorCode', 'TR.TRBCIndustryGroup', 'TR.TRBCIndustryGroupCode',
        'TR.TRBCIndustry', 'TR.TRBCIndustryCode', 'TR.IR.RatingSourceDescription.date'],
    parameters={'SDate': 0, 'EDate': -3, 'FRQ': 'FY'})  # This will look for a data-point every Fianancial Year for the past 3 Financial Years

In [26]:
# The below in this cell is needed to forward fill our dataframe correctly:
df1.replace({'': np.nan}, inplace=True)
df1.where(pd.notnull(df1), np.nan, inplace=True)

for i in df1.groupby(by=["Instrument"]):
    if i[0] == df1["Instrument"][0]: _df1 = i[1].ffill()
    else: _df1 = _df1.append(i[1].ffill())
_df1

,Instrument,Rating Source Description,Rating Source Type,Issuer Rating,TRBC Economic Sector Name,TRBC Economic Sector Code,TRBC Business Sector Name,TRBC Business Sector Code,TRBC Industry Group Name,TRBC Industry Group Code,TRBC Industry Name,TRBC Industry Code,Date
0,BARC.L,R&I Long-term Issuer Rating,RII,A,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010,2021-11-26
1,BARC.L,R&I Long-term Issuer Rating,RII,A-,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010,2020-11-18
2,BARC.L,R&I Long-term Issuer Rating,RII,A-,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010,2019-11-27
3,BARC.L,R&I Long-term Issuer Rating,RII,A-,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010,2019-11-27
4,TRI.N,NaN,NaN,NaN,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070,NaT
5,TRI.N,Moody's Senior Unsecured,MSU,Baa2,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070,2020-05-11
6,TRI.N,Egan-Jones Senior Unsecured,EJU,BBB-,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070,2019-12-02
7,TRI.N,Egan-Jones Senior Unsecured,EJU,BBB-,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070,2019-12-02


In [38]:
df2 = rd.get_history(
    universe=['VOD.L'],
    fields=['TR.IR.RatingSourceDescription', 'TR.IR.RatingSourceType',
            'TR.IR.Rating','TR.TRBCEconomicSector','TR.TRBCIndustryGroupCode'],  
    interval="1Y",
    start="2015-01-25",#T10-06",
    end="2022-02-01")

df2

VOD.L,Issuer Rating,Rating Source Description,Rating Source Type,TRBC Economic Sector Name,TRBC Industry Group Code
Date,,,,,
2015-12-17,A1,Egan-Jones Commercial Paper,EJC,,
2015-12-17,BBB+,Egan-Jones Senior Unsecured,EJU,,
2016-07-06,BBB,Egan-Jones Senior Unsecured,EJU,,
2018-10-04,BBB-,Fitch Subordinated,FBD,,
2019-11-26,BBB-,Egan-Jones Senior Unsecured,EJU,,
2021-07-29,F2,Fitch Commercial Paper,FCP,,


##### Fundamental and Reference with EDAPI

You could do the same with the [EDAPI](https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api), but as you can see, the dataframe returned is set to a format that may return many empty cells:

In [16]:
df3, err = ek.get_data(
    ['92857WAZ3=', 'BARC.L', 'TRI.N'],  # PSKc1
    fields=[
        'TR.IR.RatingSourceDescription', 'TR.IR.RatingSourceType', 'TR.GR.Rating',
        'TR.TRBCEconomicSector', 'TR.TRBCEconSectorCode', 'TR.TRBCBusinessSector',
        'TR.TRBCBusinessSectorCode', 'TR.TRBCIndustryGroup', 'TR.TRBCIndustryGroupCode',
        'TR.TRBCIndustry', 'TR.TRBCIndustryCode'],
    parameters={'SDate': 0, 'EDate': -3, 'FRQ': 'FY'})  # This will look for a data-point every Fianancial Year for the past 3 Financial Years

In [17]:
# The below in this cell is needed to forward fill our dataframe correctly:
df3.replace({'': np.nan}, inplace=True)
df3.where(pd.notnull(df3), np.nan, inplace=True)

for i in df3.groupby(by=["Instrument"]):
    if i[0] == df3["Instrument"][0]: _df3 = i[1].ffill()
    else: _df3 = _df3.append(i[1].ffill())
_df3

,Instrument,Rating Source Description,Rating Source Type,Rating,TRBC Economic Sector Name,TRBC Economic Sector Code,TRBC Business Sector Name,TRBC Business Sector Code,TRBC Industry Group Name,TRBC Industry Group Code,TRBC Industry Name,TRBC Industry Code
0,92857WAZ3=,Fitch Commercial Paper,FCP,Baa2,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020
1,92857WAZ3=,Fitch Commercial Paper,FCP,Baa2,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020
2,92857WAZ3=,Egan-Jones Senior Unsecured,EJU,BBB,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020
3,92857WAZ3=,Egan-Jones Senior Unsecured,EJU,BBB,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020
4,BARC.L,R&I Long-term Issuer Rating,RII,NaN,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
5,BARC.L,R&I Long-term Issuer Rating,RII,NaN,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
6,BARC.L,R&I Long-term Issuer Rating,RII,NaN,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
7,BARC.L,R&I Long-term Issuer Rating,RII,NaN,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
8,TRI.N,NaN,NaN,NaN,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070
9,TRI.N,Moody's Senior Unsecured,MSU,NaN,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070



##### Fundamental and Reference with `rd.content.fundamental_and_reference.Definition`

You could also get this data with `content.fundamental_and_reference.Definition`:

In [42]:
df4 = rd.content.fundamental_and_reference.Definition(
    universe=['GB047986974=', 'BARC.L', 'TRI.N'],
    fields=['TR.IR.RatingSourceDescription', 'TR.IR.RatingSourceType', 'TR.GR.Rating',
        'TR.TRBCEconomicSector', 'TR.TRBCEconSectorCode', 'TR.TRBCBusinessSector',
        'TR.TRBCBusinessSectorCode', 'TR.TRBCIndustryGroup', 'TR.TRBCIndustryGroupCode',
        'TR.TRBCIndustry', 'TR.TRBCIndustryCode']
).get_data().data.df

df4

,Instrument,Rating Source Description,Rating Source Type,Rating,TRBC Economic Sector Name,TRBC Economic Sector Code,TRBC Business Sector Name,TRBC Business Sector Code,TRBC Industry Group Name,TRBC Industry Group Code,TRBC Industry Name,TRBC Industry Code
0,GB047986974=,Egan-Jones Commercial Paper,EJC,NR,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020
1,BARC.L,Egan-Jones Senior Unsecured,EJU,<NA>,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
2,TRI.N,Egan-Jones Commercial Paper,EJC,<NA>,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070


Note how the fields that are inappropriate for the instrument asked (e.g.: `TRBC Economic Sector Code` for `BARC.L`) show up as `<NA>`, allowing you to mix and match fields and instruments even if they are not all complementary to each other, and collect all the information you are after, for all instruments you may me interested in, in one call.

## AdfinX RealTime - AdxRtChain

### What does this do?

As per [Tutorial 3 - Real-time Chain Retrieval - AdxRtChain](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-3-real-time-chain-retrieval-adx-rt-chain)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/adxrtchain.zip), Adfin X RealTime Chain (AdxRtChain) returns a list of the constituent instrument codes for any chain such as 0#.FTSE (the FTSE 100 instruments). Data is returned as OnUpdate event, the only other event is OnStatusChange:

<img src="imgs/AdxRtChain.PNG">

### VBA

In VBA, we went through with the creation of `cmdGetChain_Click`:

>     Private Sub cmdGetChain_Click()
>         ActiveCell.Select
>         
>         If myAdxRtChain Is Nothing Then Set myAdxRtChain = CreateAdxRtChain()
>         
>         With myAdxRtChain
>             .Source = "IDN"
>             .ItemName = Range("G6").Value
>             .RequestChain
>         End With
>     End Sub

then `myAdxRtChain_OnUpdate`:

>     Private Sub myAdxRtChain_OnUpdate(ByVal DataStatus As AdfinXRtLib.RT_DataStatus)
>         Dim i As Integer
>         
>         If DataStatus = RT_DS_FULL Then
>             For i = 1 To UBound(myAdxRtChain.Data)
>                 Range("G8").Offset(i - 1, 0).Value = myAdxRtChain.Data(i)
>             Next i
>         End If
>     End Sub

then we ought to make sure we can close the connection with `cmdClearChain_Click`, which is done simply in Python with `rd.close_session()`.

We can replicate this easily in Python with the Pricing snapshots and Fundamental & Reference data function `get_data()`:

### Python

In [33]:
FTSEConstituentDf1 = rd.get_data(
    universe=['0#.FTSE'], # in Office COM API, you do not need the 0#, but it is requiered on our Python API calls
    fields=['TR.TURNOVER.timestamp', 'TR.TURNOVER', 'TR.EVToSales'])
FTSEConstituentDf1

,Instrument,Timestamp,Turnover,Enterprise Value To Sales (Daily Time Series Ratio)
0,STAN.L,2022-09-13T00:00:00Z,5.101301e+09,6.603833
1,CRDA.L,2022-09-13T00:00:00Z,2.035870e+09,4.813461
2,ANTO.L,2022-09-13T00:00:00Z,2.557363e+09,2.683328
3,BNZL.L,2022-09-13T00:00:00Z,1.712938e+09,1.046692
4,SGE.L,2022-09-13T00:00:00Z,1.748899e+09,4.441644
...,...,...,...,...
95,EDV.L,2022-09-13T00:00:00Z,5.006498e+08,1.984468
96,OCDO.L,2022-09-13T00:00:00Z,3.611811e+09,2.694019
97,LSEG.L,2022-09-13T00:00:00Z,4.255721e+09,7.101401
98,TSCO.L,2022-09-13T00:00:00Z,4.740771e+09,0.472315


In [45]:
FTSEConstituentDf2, err = ek.get_data(
    ['0#LCO:'],  # PSKc1
    fields=['CF_ASK', 'CF_BID','CF_LAST','SETTLE','OPINT_1'])
FTSEConstituentDf2

,Instrument,CF_ASK,CF_BID,CF_LAST,SETTLE,OPINT_1
0,LCOTOT,<NA>,<NA>,<NA>,<NA>,2425742
1,LCOZ2,93.74,93.72,93.73,92.41,323850
2,LCOF3,91.8,91.78,91.81,90.56,411523
3,LCOG3,89.99,89.96,89.99,88.85,214347
4,LCOH3,88.46,88.44,88.42,87.42,135786
...,...,...,...,...,...,...
72,LCOX8,<NA>,<NA>,68.42,68.42,<NA>
73,LCOZ8,<NA>,67.58,68.32,68.32,124
74,LCOF9,<NA>,<NA>,68.25,68.25,<NA>
75,LCOG9,<NA>,<NA>,68.19,68.19,<NA>


# Add to appendix - replace with TR.F fields

You could also use the [Fundamental and Reference](https://github.com/Refinitiv-API-Samples/Example.DataLibrary.Python/blob/main/Examples/2-Content/2.09-FundamentalAndReference/EX-2.09.01-FundamentalAndReference.ipynb) to get such constituent data:

In [50]:
FTSEConstituent = rd.content.fundamental_and_reference.Definition(
    ["0#.FTSE"],
    ["TR.TURNOVER.date","TR.TURNOVER", "TR.NetIncome.date","TR.NetIncome"]
).get_data()

SPTTTKConstituent.data.df

,Instrument,Date,Turnover,Date,Net Income Incl Extra Before Distributions
0,STAN.L,2022-10-19,1.852077e+09,2021-12-31,2315000000
1,CRDA.L,2022-10-19,2.088025e+09,2021-12-31,320800000
2,ANTO.L,2022-10-19,1.248494e+09,2021-12-31,1290200000
3,BNZL.L,2022-10-19,2.153450e+09,2021-12-31,442800000
4,SGE.L,2022-10-19,1.177236e+09,2021-09-30,285000000
...,...,...,...,...,...
95,EDV.L,2022-10-19,4.160945e+08,2021-12-31,215500000
96,OCDO.L,2022-10-19,1.034875e+09,2021-11-28,-223200000
97,LSEG.L,2022-10-19,2.001095e+09,2021-12-31,3129000000
98,TSCO.L,2022-10-19,1.925327e+09,2022-02-26,1481000000


## AdfinX RealTime - AdxRtHistory - Interday Time Series History

### What does this do? 

Adfin RealTime History (AdxRtHistory) is used to retrieve time series (historic) data for an instrument or instruments except for intraday data. This was best exemplified in [Tutorial 5 - Time Series History - AdxRtHistory](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-5-time-series-history-adx-rt-history)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/adxrthistory.zip):


<img src="imgs/AdxRtHistory Interday Time Series History Eg.PNG">

### VBA

In VBA, we used `AdfinXRtLib`:

>     ' Note the use of CreateReutersObject - function in the PLVbaApis module.
>         If myAdxRtHist Is Nothing Then Set myAdxRtHist = CreateReutersObject("AdfinXRtLib.AdxRtHistory")
>         
>         On Error GoTo errHndlr
>         With myAdxRtHist
>             .FlushData
>             .ErrorMode = EXCEPTION ' EXCEPTION, DialogBox, NO_EXCEPTION
>             .Source = "IDN"
>             .ItemName = [C7].Value
>             .Mode = [H8].Value
>             .RequestHistory ("DATE,CLOSE,VOLUME") 'NOTE USE OF OLD FIELD NAMES, NOT ("TRDPRC_1.TIMESTAMP,TRDPRC_1.CLOSE,TRDPRC_1.VOLUME")
>             
>             'arrFlds = Array("DATE","CLOSE","VOLUME")
>             '.RequestHistory ()arrFlds
>             '.RequestHistory ("*") ' "*" requests all fields.
>         End With

before the `Private Sub` '`myAdxRtHist_OnUpdate(ByVal DataStatus As AdfinXRtLib.RT_DataStatus)`'.

Things are simpler in Python:

### Python

As aforementioned, AdxRtHistory is used to retrieve time series (historic) data for an instrument or instruments except for intraday data. This is exactly what the instruments `get_history` is for!

In [36]:
SPTTTKConstituents = list(SPTTTKConstituent.data.df['Instrument'])
print(SPTTTKConstituents)

['BB.TO', 'HAI.TO', 'ENGH.TO', 'ALYA.TO', 'SHOP.TO', 'SW.TO', 'GIBa.TO', 'LSPD.TO', 'NVEI.TO', 'KXS.TO', 'SYZ.TO', 'TCS.TO', 'ABST.TO', 'CTS.TO', 'QTRH.TO', 'HUT.TO', 'CSU.TO', 'TIXT.TO', 'SFTC.TO', 'MAGT.TO', 'OTEX.TO', 'ET.TO', 'CLS.TO', 'DSG.TO', 'DND.TO']


In [37]:
TimeSeriesDf = rd.get_history(
    universe=SPTTTKConstituents[1:6],
    fields=['TR.RevenueMean', 'TR.PriceClose'],  # 'TR' fields are usually historic ones.
    interval="1D",
    start="2022-01-25",
    end="2022-02-01")

In [38]:
TimeSeriesDf

HAI.TO                    ENGH.TO              \
           Revenue - Mean Price Close Revenue - Mean Price Close   
Date                                                               
2021-09-15      129230000                       <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-12-03           <NA>        <NA>           <NA>        <NA>   
2021-12-03           <NA>        <NA>           <NA>        <NA>   
2021-12-03           <NA>        <NA>           <NA>        <NA>   
2021-12-03           <NA>        <NA>           <NA>        <NA>   
2021-12-03           <NA>        <NA>           <NA>        <NA>   
2021-12-03           <NA>        <NA>           <NA>        <NA>   
2021-12-17           <NA>        <NA>      482567500               
2021-12-17           <NA>        <NA>      482567500               
2021-12-17           <NA>        <NA>      482567500               
2021-12-17           <NA>        <NA>      482567500               
2022-01-25                        6.9                      41.94   
2022-01-26      125700500        6.52                      42.68   
2022-01-26      125700500                       <NA>        <NA>   
2022-01-26      125700500                       <NA>        <NA>   
2022-01-26      125700500                       <NA>        <NA>   
2022-01-26      125700500                       <NA>        <NA>   
2022-01-27                       6.37                      42.28   
2022-01-28                       5.97                         43   
2022-01-30           <NA>        <NA>      485067500               
2022-01-30           <NA>        <NA>      485067500               
2022-01-31                       5.99                      44.78   
2022-02-01                          6                      44.33   

                  ALYA.TO                    SHOP.TO              \
           Revenue - Mean Price Close Revenue - Mean Price Close   
Date                                                               
2021-09-15           <NA>        <NA>           <NA>        <NA>   
2021-11-12      416484170                       <NA>        <NA>   
2021-11-12      416484170                       <NA>        <NA>   
2021-11-12      416484170                       <NA>        <NA>   
2021-11-12      416484170                       <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12           <NA>        <NA>           <NA>        <NA>   
2021-11-12      416484170                       <NA>        <NA>   
2021-12-03           <NA>        <NA>     4563826080               
2021-12-03           <NA>        <NA>     4563826080               
2021-12-03           <NA>        <NA>     4563826080               
2021-12-03           <NA>        <NA>     4563826080               
2021-12-03           <NA>        <NA>     4563826080               
2021-12-03           <NA>        <NA>     4563826080               
2021-12-17           <NA>        <NA>           <NA>        <NA>   
2021-12-17           <NA>        <NA>           <NA>        <NA>   
2021-12-17           <NA>        <NA>           <NA>        <NA>   
2021-1

## AdxRtHistory - Intraday Time Series History

### VBA

Adfin RealTime History (AdxRtHistory) Intraday is similar:



Which had few VBA lines needed:

>     Private Sub cmdGetInterday_Click()
>         ActiveCell.Select
>         
>         MsgBox "AdxRtHistory cannot retrieve INTRA day data, use the RHistoryAPI instead"
>     End Sub


### Python

Intraday data is just as easy to get:

In [39]:
# rd.open_session("platform.rdp")

In [40]:
IntradayTimeSeriesDf = rd.get_history(
    universe=SPTTTKConstituents,
    fields=['TRDPRC_1'],
    interval="1min",  # The consolidation interval. Supported intervals are: tick, tas, taq, minute, 1min, 5min, 10min, 30min, 60min, hourly, 1h, daily, 1d, 1D, 7D, 7d, weekly, 1W, monthly, 1M, quarterly, 3M, 6M, yearly, 1Y.
    start="2022-06-01T13:00:00",
    end="2022-06-01T16:00:00")

In [41]:
IntradayTimeSeriesDf

TRDPRC_1,BB.TO,HAI.TO,ENGH.TO,ALYA.TO,SHOP.TO,SW.TO,GIBa.TO,LSPD.TO,NVEI.TO,KXS.TO,...,HUT.TO,CSU.TO,TIXT.TO,SFTC.TO,MAGT.TO,OTEX.TO,ET.TO,CLS.TO,DSG.TO,DND.TO
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-06-01 13:30:00,7.71,5.12,33.99,2.93,48.468,28.25,108.54,33.9,65.35,142.04,...,3.28,2019.98,31.99,<NA>,16.41,52.52,<NA>,14.13,75.06,23.27
2022-06-01 13:31:00,7.72,<NA>,<NA>,<NA>,48.76,28.7,108.75,33.86,65.99,<NA>,...,3.33,<NA>,<NA>,<NA>,<NA>,52.61,<NA>,<NA>,<NA>,<NA>
2022-06-01 13:32:00,7.7,<NA>,<NA>,<NA>,48.993,28.57,108.79,33.8,66.25,<NA>,...,3.32,2032.63,<NA>,<NA>,<NA>,52.67,<NA>,<NA>,<NA>,<NA>
2022-06-01 13:33:00,7.7,<NA>,<NA>,<NA>,49.1,28.87,108.7,33.58,66.17,<NA>,...,3.32,2013.69,<NA>,<NA>,<NA>,52.56,<NA>,<NA>,75.92,<NA>
2022-06-01 13:34:00,7.75,<NA>,34.21,<NA>,48.983,28.99,108.77,33.79,66.13,<NA>,...,3.34,2025.89,<NA>,<NA>,16.46,52.52,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-01 15:55:00,<NA>,<NA>,33.67,<NA>,45.897,<NA>,<NA>,31.66,61.79,<NA>,...,3.01,<NA>,31.36,<NA>,<NA>,51.39,<NA>,13.81,<NA>,22.07
2022-06-01 15:56:00,<NA>,<NA>,<NA>,<NA>,45.838,<NA>,107.36,31.67,61.87,<NA>,...,3.01,<NA>,31.35,22.07,<NA>,51.4,<NA>,13.81,<NA>,<NA>
2022-06-01 15:57:00,7.25,<NA>,<NA>,<NA>,45.781,<NA>,107.4,31.51,61.8,<NA>,...,3,<NA>,<NA>,<NA>,<NA>,51.38,<NA>,13.81,<NA>,22.03


## RHistory

### What does this do?

As per the [EIKON FOR DEVELOPERS](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/documentation/eikon_for_developers_09032016.pdf) documentation that you may find [here](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/documentation),

    The RHistory API enables developers to access time series data in VBA inside Excel in the same way as the RHistory function. The RHistory function retrieves a list of time series data for one instrument or a list of instruments at regular intervals (for example, on a daily, weekly, monthly, and yearly basis) for a given time period or for a given number of records. It also provides time series data at non-regular intervals, for example, TAS (Time and Sales), TAQ (Trade and Quote), and TICK (tick by tick).
    
This COM API is best exemplified in [Tutorial 9 - Time Series History - RHistoryAPI](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-9-time-series-history-r-history-api)'s [Excxel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/rhistoryapi.zip) in which you will find the following VBA:
    
### VBA

In VBA, we declared `myRHistoryQuery` via `Private WithEvents myRHistoryQuery As RHistoryAPI.RHistoryQuery` before:

>         ' Set the query parameters accordingly to your needs
>         With myRHistoryQuery
>             .InstrumentIdList = Range("G6").Value
>             .FieldList = Range("G7").Value 'Or of the form  "TRDPRC_1.TIMESTAMP;TRDPRC_1.VALUE;TRDPRC_1.VOLUME"
>             
>             '.FieldList = "TRDPRC_1.TIMESTAMP;TRDPRC_1.HIGH;TRDPRC_1.CLOSE;TRDPRC_1.LOW;TRDPRC_1.OPEN;TRDPRC_1.VOLUME;TRDPRC_1.COUNT"
>             .RequestParams = Range("G8").Value
>             .RefreshParams = Range("G9").Value
>             .DisplayParams = Range("G10").Value
>         
>             .Subscribe
>         End With

This allowed us to run sheets such as:

<img src="imgs/RHistory Eg.PNG">


### Python

For this, we can (yet again) use the `get_history` function:

In [42]:
IntradayTimeSeriesDf2 = rd.get_history(
    universe=SPTTTKConstituents,
    fields=['TRDPRC_1'],
    interval="10min", # The consolidation interval. Supported intervals are: tick, tas, taq, minute, 1min, 5min, 10min, 30min, 60min, hourly, 1h, daily, 1d, 1D, 7D, 7d, weekly, 1W, monthly, 1M, quarterly, 3M, 6M, yearly, 1Y.
    start="2022-06-01T13:00:00",
    end="2022-06-01T16:00:00")

In [43]:
IntradayTimeSeriesDf2

TRDPRC_1,BB.TO,HAI.TO,ENGH.TO,ALYA.TO,SHOP.TO,SW.TO,GIBa.TO,LSPD.TO,NVEI.TO,KXS.TO,...,HUT.TO,CSU.TO,TIXT.TO,SFTC.TO,MAGT.TO,OTEX.TO,ET.TO,CLS.TO,DSG.TO,DND.TO
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-06-01 13:30:00,7.60,5.52,34.21,2.9,48.896,28.85,108.48,33.30,65.58,141.720,...,3.270,2020.53,32.01,22.55,16.20,52.24,<NA>,13.93,76.15,23.270
2022-06-01 13:40:00,7.68,5.6,<NA>,<NA>,49.350,29.02,108.40,33.25,65.55,143.110,...,3.220,2010.57,31.97,22.49,16.41,52.16,<NA>,13.95,76.34,23.800
2022-06-01 13:50:00,7.65,<NA>,<NA>,<NA>,49.020,28.93,108.71,33.33,64.35,142.890,...,3.230,2003.25,31.95,22.29,16.40,52.25,13.79,13.91,76.30,23.445
2022-06-01 14:00:00,7.56,5.7,34.28,2.92,48.509,29.03,107.81,32.96,62.88,141.890,...,3.140,1989.87,31.75,22.25,16.12,51.88,13.75,13.86,76.04,23.160
2022-06-01 14:10:00,7.44,5.66,33.91,2.92,47.808,29.00,107.71,32.64,62.80,141.330,...,3.080,1978.31,31.58,22.11,16.12,51.63,<NA>,13.75,75.56,22.860
2022-06-01 14:20:00,7.45,<NA>,34.2,2.92,48.373,28.93,107.74,32.95,62.71,141.750,...,3.090,1977.42,31.62,22.16,15.95,51.59,<NA>,13.76,76.00,22.940
2022-06-01 14:30:00,7.41,5.62,33.77,<NA>,47.747,28.81,107.71,32.85,62.36,141.050,...,3.075,1972.29,31.54,<NA>,15.81,51.43,<NA>,13.75,75.84,22.560
2022-06-01 14:40:00,7.44,<NA>,33.87,<NA>,47.600,29.13,107.66,32.67,62.59,140.510,...,3.080,1975.77,31.47,<NA>,15.88,51.64,<NA>,13.75,76.25,22.690
2022-06-01 14:50:00,7.47,<NA>,33.95,2.94,48.402,29.18,107.92,32.74,63.11,141.630,...,3.120,1984.9,31.58,<NA>,15.86,51.72,<NA>,13.86,76.43,22.670


## PLSynchronization Manager

This synchronisation manager was an old method whereby the COM API managed connections to Refinitiv Datasets. It is no longer required in python.

## DEX2

### What does this do?

The DEX2 interface provided a broad range of content and functionality such Fundamental and Reference as well as timeseries histories of such fields. The DEX2 interface also provided interday timeseries histories for both realtime and non-realtime data. In addition you could use it for symbology conversion. 

### DEX2 ID to RIC (Symbology Conversion)


As shown in [Tutorial 6 - Data Engine - Dex2](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-6-data-engine-dex-2)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/dex2.zip), we can convert symbols using the COM API.

#### VBA

After setting `MyDex2Mgr = CreateReutersObject("Dex2.Dex2Mgr")`, for ISINs, we can then go ahead with

>     ' Must call Initialise() at once
>     m_cookie = MyDex2Mgr.Initialize()
>     ' We can then create an RData object
>     Set myRData1 = MyDex2Mgr.CreateRData(m_cookie)
>     
>     With MyDex2Mgr
>         .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>     End With
>     
>     ' Set the properties & methods for the DEX2Lib.RData object
>     With myRData1
>         .InstrumentIDList = Range("B7").Value ' Could use "TRI.N", "TRI.N; GOOG.O; MSFT.O" or array, e.g. arrInstrList.
>         .FieldList = Range("D7").Value & "; " & Range("D8").Value  '"RI.ID.RIC; RI.ID.ISIN" ' Could use "RI.ID.RIC", "RI.ID.RIC; RI.ID.WERT" or array, e.g. arrFldList.
>         .DisplayParam = "Transpose:Y"
>         'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>             "", "RH:In CH:Fd Transpose:Y"
>         'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>         
>         'Ignore cache; get data directly from the Snapshot Server
>         '.Subscribe False
>         'Or use cache by default
>         .Subscribe
>     End With
>     Exit Sub

For SEDOL, we can use:

>     ' Must call Initialise() at once
>     m_cookie = MyDex2Mgr.Initialize()
>     ' We can then create an RData object
>     Set myRData2 = MyDex2Mgr.CreateRData(m_cookie)
>     
>     With MyDex2Mgr
>         .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>     End With
>     
>     ' Set the properties & methods for the DEX2Lib.RData object
>     With myRData2
>         .InstrumentIDList = [B10].Value ' Could use "TRI.N", "TRI.N; GOOG.O; MSFT.O" or array, e.g. arrInstrList.
>         .FieldList = Range("D10").Value & "; " & Range("D11").Value  '"RI.ID.RIC; RI.ID.SEDOL" ' Could use "RI.ID.RIC", "RI.ID.RIC; RI.ID.WERT" or array, e.g. arrFldList.
>         .DisplayParam = "Transpose:Y"
>         'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>             "", "RH:In CH:Fd Transpose:Y"
>         'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>         
>         'Ignore cache; get data directly from the Snapshot Server
>         '.Subscribe False
>         'Or use cache by default
>         .Subscribe
>     End With
>     Exit Sub

#### Python

Symbology conversions in python are a lot more powerful than in Excel using the COM API. The extent of the use-cases for symbology conversions are best explained in our [GitHub Repository](https://github.com/Refinitiv-API-Samples/Example.DataLibrary.Python/blob/main/Examples/2-Content/2.05-Symbology/EX-2.05.01-Symbology.ipynb), but for the sake of completeness, you can see an example use in the 2 cells directly below:

In [44]:
# # Get generic table of many symbols for 4 companies of choice:
response = rd.content.symbol_conversion.Definition(symbols=["MSFT.O", "AAPL.O", "GOOG.O", "IBM.N"]).get_data()
response.data.df

,DocumentTitle,RIC,IssueISIN,CUSIP,SEDOL,TickerSymbol,IssuerOAPermID
MSFT.O,"Microsoft Corp, Ordinary Share, NASDAQ Global ...",MSFT.O,US5949181045,594918104,<NA>,MSFT,4295907168
AAPL.O,"Apple Inc, Ordinary Share, NASDAQ Global Selec...",AAPL.O,US0378331005,037833100,<NA>,AAPL,4295905573
GOOG.O,"Alphabet Inc, Ordinary Share, Class C, NASDAQ ...",GOOG.O,US02079K1079,02079K107,<NA>,GOOG,5030853586
IBM.N,"International Business Machines Corp, Ordinary...",IBM.N,US4592001014,459200101,2005973,IBM,4295904307


In [45]:
# # Convert specific symbols:
response = rd.content.symbol_conversion.Definition(
    symbols=["US5949181045", "US02079K1079"],
    from_symbol_type=rd.content.symbol_conversion.SymbolTypes.ISIN, # Symbol types: RIC => RIC; ISIN => IssueISIN; CUSIP => CUSIP; SEDOL => SEDOL; TICKER_SYMBOL => TickerSymbol; OA_PERM_ID => IssuerOAPermID; LIPPER_ID => FundClassLipperID
    to_symbol_types=[rd.content.symbol_conversion.SymbolTypes.RIC,
                     rd.content.symbol_conversion.SymbolTypes.OA_PERM_ID],
).get_data()

response.data.df

,DocumentTitle,RIC,IssuerOAPermID
US5949181045,"Microsoft Corp, Ordinary Share, NASDAQ Global ...",MSFT.O,4295907168
US02079K1079,"Alphabet Inc, Ordinary Share, Class C, NASDAQ ...",GOOG.O,5030853586


### DEX2 RData Equity Data

As best exemplified in [Tutorial 6 - Data Engine - Dex2](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-6-data-engine-dex-2)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/dex2.zip), the DEX2 RData Equity Data allows us to access historical [Market Capital](https://www.investopedia.com/terms/m/marketcapitalization.asp):

<img src="imgs/Dex2Equity.PNG">
# take this generic stuff down into appendix and make sure all components are referenced

#### VBA

DEX2 replicates RData() and TR() functions for real time data retrieval in Eikon for Excel / Eikon - Microsoft Office. The DEX2 methods differ slightly in the initialisation and use of the Dex2Mgr object depending upon whether RData() or the TR() function is being replicated. The examples below explain this. The TR() function can be used to retrieve data from ADC (the Analytics Data Cloud) and this is contained in Tutorial 11 - Dex2 TR function Analytics Data Cloud ADC data here.

NOTE - Eikon for Excel or Eikon - Microsoft Office should be logged in otherwise the sample VBA code will generate an error "ERROR #360c - AdxRtx : No connection to the platform".

##### Prerequisites
1. Open a new single sheet Excel workbook. Save As with an appropriate name (e.g. DEX2.xls or DEX2.xlsm in Office 2007 or higher).
2. Go to the VBE (Visual Basic Editor), ensure the Project Explorer is visible and select the project for the workbook above. \<ALT\>\<F11\> or Tools, Macro, Visual Basic Editor in Excel 2003 or  Developer, Visual Basic in Excel 2007 and above, View, Project Explorer If the Developer header is not visible in Excel 2007 and above, go to the Excel Office Button, select Excel Options (lower right), Popular, and check the 'Show Developer tab in the Ribbon' box.
3. In the VBE, click on File, Import File and import PLVbaApis.bas. The .bas location is C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin (Z may be X or Y, depending on the last Eikon update). The .bas is loaded as a new VB project module, PLVbaApis.
4. In the PLVbaAPis module, comment out the sections which aren't required. As this example is dealing with DEX2, part of the library DEX2Lib, the DEX2 section can remain uncommented.
5. In the VBE, go Tools, References and ensure that the DEX2 Library is checked. If it is not in the list the library is called DEX2.dll and its location for Eikon 4 is;

    "C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin" (Z may be X or Y, depending on the last Eikon update).

    Note that for the latest versions of Eikon the default path is;

    "C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin\Apps\TR.OFFICE.CORE\0.0.0.0\Bin" (Z may be X or Y, depending on the last Eikon update).

##### DEX2

6. Create an instance of an DEX2Mgr object using the PLVbaApis function CreateDex2Mgr().

    Set myDEX2Mgr = CreateDex2Mgr()
 
7. To replicate RData(), create the Dex2Mgr object (myDex2Mgr) and initialise it. Use the return value from the initialisation to create an instance of the Dex2Lib.RData Class (myDex2RData) using the CreateRData method.

8. Now go ahead with `cmdMktCapHist_Click` creation:

>    Private Sub cmdMktCapHist_Click()
>        On Error GoTo errHandler
>        
>        ActiveCell.Select
>        
>        Range("F20:G100").ClearContents
>        If MyDex2Mgr Is Nothing Then Set MyDex2Mgr = CreateReutersObject("Dex2.Dex2Mgr")
>        
>        ' Must call Initialise() at once
>        m_cookie = MyDex2Mgr.Initialize()
>        ' We can then create an RData object
>        Set myRData2 = MyDex2Mgr.CreateRData(m_cookie)
>        
>        With MyDex2Mgr
>            .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>        End With
>        
>        ' Set the properties & methods for the DEX2Lib.RData object.
>        With myRData2
>            .InstrumentIDList = [C15].Value
>            .FieldList = [C16].Value '"RII.A.INDEX_MKT_CAP_RTRS" '"RII.A.INDEX_MKT_CAP_RTRS; RII.A.INDEX_MKT_CAP_USD_RTRS"
>            ' Could use single "RI.ID.RIC", multiple "RI.ID.RIC; RI.ID.WERT" or array, e.g. arrFldList.
>            .RequestParam = [C17].Value
>            .DisplayParam = "RH:D CH:Fd SORT:DESC"
>            'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>                "", "RH:In CH:Fd Transpose:Y"
>            'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>            
>            'Ignore cache; get data directly from the Snapshot Server
>            '.Subscribe False
>            'Or use cache by default
>            .Subscribe
>        End With
>        Exit Sub
>        
>    errHandler:
>        MsgBox MyDex2Mgr.GetErrorString(Err.Number)
>    End Sub
>    
>    ' OnUpdate event callback for myRData1
>    Private Sub myRData2_OnUpdate(ByVal DataStatus As Dex2Lib.DEX2_DataStatus, ByVal Error As Variant)
>        Dim C As Integer, r As Integer
>        Dim res2 As Variant
>        
>        If Error <> 0 Then [F21].Value = Error: Exit Sub
>    
>        res2 = myRData2.Data
>        
>        If IsEmpty(res2) Then [F21].Value = "No data": Exit Sub
>        
>        ' r for the rows
>        For r = LBound(res2, 1) To UBound(res2, 1)
>            ' c for the columns
>            For C = LBound(res2, 2) To UBound(res2, 2)
>                [F21].Offset(r, C).Value = res2(r, C)
>            Next C
>        Next r
>    End Sub

#### Python

Couldn't be easier in Python:

In [ ]:
MarketCapDf = rd.get_history(
    universe=['PEUP.PA'],
    fields=['TR.MARKETCAPITALISATION'],
    interval="1d", # The consolidation interval. Supported intervals are: tick, tas, taq, minute, 1min, 5min, 10min, 30min, 60min, hourly, 1h, daily, 1d, 1D, 7D, 7d, weekly, 1W, monthly, 1M, quarterly, 3M, 6M, yearly, 1Y.
    start="2022-08-10",
    end="2022-09-13")
MarketCapDf

### DEX2 Time Series

DEX2 returns reference and fundamental data for instruments. Data includes identifiers such as ISIN and Cusip codes for an instrument, market capitalisation history, business sector information, ratings information and time series data. This was best exemplified in [Tutorial 6 - Data Engine - Dex2](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-6-data-engine-dex-2)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/dex2.zip):

<img src="imgs/DEX2 TR Data.PNG">

#### VBA

You'd have to Initialize Dex2 session under the sub `cmdInitialize_Click()`:

>    ' Initialize Dex2 session when the data request is first started.
>    Private Sub cmdInitialize_Click()
>        On Error GoTo ErrorHandle
>    
>        ActiveCell.Select
>    
>        ' Clear the output of logger
>        lbLog.Clear
>    
>        ClearAllOutput
>    
>        ' Clear the Cookie and old query
>        If Not MyDex2Cookie = 0 Then
>            MyDex2Mgr.Finalize (MyDex2Cookie)
>            MyDex2Cookie = 0
>            Set MyDex2Mgr = Nothing
>            Set MyDex2RData = Nothing
>        End If
>        
>        ' Instantiate the Dex2 manager
>        Set MyDex2Mgr = CreateDex2Mgr()
>        Set MyDex2MgrADC = MyDex2Mgr
>        ' Instantiate the RSearch logger
>        Set MyDex2Logger = New CLogger
>        
>        ' Initialize Dex2 session
>        MyDex2Cookie = MyDex2MgrADC.Initialize(DE_MC_ADC_POWERLINK, MyDex2Logger)
>            
>        ' We can choose to display error code
>        MyDex2Mgr.SetErrorHandling MyDex2Cookie, DE_EH_ERROR_CODES
>        ' Or display error description
>        ' MyDex2Mgr.SetErrorHandling MyDex2Cookie, DE_EH_STRING
>            
>        ' Create a Dex2 query using the session cookie
>        Set MyDex2RData = MyDex2Mgr.CreateRData(MyDex2Cookie)
>        
>        ' Create a Dex2 query manager using the session cookie
>        Set MyDex2RDataMgr = MyDex2Mgr.CreateRDataMgr(MyDex2Cookie)
>        
>        Exit Sub

#### Python

In Python, things are a little simpler:


In [51]:
DEX2Df = rd.get_history(
    universe=['.FCHI', 'BARC.L', 'TRI.N'],
    fields=['TR.Index_MKT_CAP_RTRS', 'TR.Close'],
    interval="1D",
    start="2021-10-01",
    end="2021-10-11")

In [47]:
pd.set_option('display.max_columns', None)  # This allows us to see all the columsn of the returned data-frame

In [48]:
DEX2Df

.FCHI              \
           Calculated Index Market Capitalisation Price Close   
Date                                                            
2021-10-01                           2.135779e+12     6517.69   
2021-10-04                           2.121481e+12     6477.66   
2021-10-05                           2.152681e+12     6576.28   
2021-10-06                           2.125950e+12     6493.12   
2021-10-07                           2.163303e+12     6600.19   
2021-10-08                           2.147288e+12     6559.99   
2021-10-11                           2.151504e+12     6570.54   

                                           BARC.L              \
           Calculated Index Market Capitalisation Price Close   
Date                                                            
2021-10-01                                   <NA>      187.88   
2021-10-04                                   <NA>      187.28   
2021-10-05                                   <NA>      194.60   
2021-10-06                                   <NA>      191.90   
2021-10-07                                   <NA>      193.74   
2021-10-08                                   <NA>      195.16   
2021-10-11                                   <NA>      197.68   

                                            TRI.N              
           Calculated Index Market Capitalisation Price Close  
Date                                                           
2021-10-01                                   <NA>      110.16  
2021-10-04                                   <NA>      109.42  
2021-10-05                                   <NA>      110.43  
2021-10-06                                   <NA>      111.77  
2021-10-07                                   <NA>      113.64  
2021-10-08                                   <NA>      113.90  
2021-10-11                                   <NA>      114.37

### DEX2 RData Treasury Data

This functionality was best shown in [Tutorial 6 - DEX2 Treasury](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-6-data-engine-dex-2) where Treasury Ratings and their Source Descriptions were discoverable:

<img src="imgs/DEX2 RData Treasury Data.PNG">

#### VBA

>     Private Sub cmdClearAll_Click()
>         ActiveCell.Select
>         
>         Range("G10").Value = ""
>         Range("G21:J100").Value = ""
>         
>         If Not myRData1 Is Nothing Then Set myRData1 = Nothing
>         If Not myRData2 Is Nothing Then Set myRData2 = Nothing
>         If Not MyDex2Mgr Is Nothing Then Set MyDex2Mgr = Nothing
>     End Sub
>     
>     Private Sub cmdGetRating_Click()
>         On Error GoTo errHandler
>         
>         ActiveCell.Select
>         
>         [G7].Value = ""
>         ' Note the use of CreateReutersObject - function in the PLVbaApis module.
>         If MyDex2Mgr Is Nothing Then Set MyDex2Mgr = CreateReutersObject("Dex2.Dex2Mgr")
>         
>         ' Must call Initialise() at once
>         m_cookie = MyDex2Mgr.Initialize()
>         ' We can then create an RData object
>         Set myRData1 = MyDex2Mgr.CreateRData(m_cookie)
>         
>         With MyDex2Mgr
>             .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>         End With
>         
>         ' Set the properties & methods for the DEX2Lib.RData object
>         With myRData1
>             .InstrumentIdList = [C6].Value ' Could use "TRI.N", "TRI.N; GOOG.O; MSFT.O" or array, e.g. arrInstrList.
>             .FieldList = [C7].Value '"EJV.GR.Rating" ' Could use "RI.ID.RIC", "RI.ID.RIC; RI.ID.WERT" or array, e.g. arrFldList.
>             .RequestParam = [C8].Value '"RTSRC:S&P"
>             'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>                 "", "RH:In CH:Fd Transpose:Y"
>             'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>             
>             'Ignore cache; get data directly from the Snapshot Server
>             '.Subscribe False
>             'Or use cache by default
>             .Subscribe
>         End With
>         Exit Sub
>         
>     errHandler:
>         MsgBox MyDex2Mgr.GetErrorString(Err.Number)
>     End Sub
>     
>     ' OnUpdate event callback for myRData1
>     Private Sub myRData1_OnUpdate(ByVal DataStatus As Dex2Lib.DEX2_DataStatus, ByVal Error As Variant)
>         Dim res As Variant
>     
>         'Debug.Print DataStatus
>     
>         If Error <> 0 Then [G10].Value = Error: Exit Sub
>     
>         ' get the data retrieved from the database
>         res = myRData1.Data
>     
>         ' Display the result.
>         [G10].Value = res
>     End Sub
>     
>     Private Sub cmdGetHistoryOfRating_Click()
>         On Error GoTo errHandler
>         
>         ActiveCell.Select
>         
>         Range("G21:J100").ClearContents
>         If MyDex2Mgr Is Nothing Then Set MyDex2Mgr = CreateReutersObject("Dex2.Dex2Mgr")
>         
>         ' Must call Initialise() at once
>         m_cookie = MyDex2Mgr.Initialize()
>         ' We can then create an RData object
>         Set myRData2 = MyDex2Mgr.CreateRData(m_cookie)
>         
>         With MyDex2Mgr
>             .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>         End With
>         
>         ' Set the properties & methods for the DEX2Lib.RData object
>         With myRData2
>             .InstrumentIdList = [C15].Value
>             .FieldList = [C16].Value
>             .RequestParam = [C17].Value
>             .DisplayParam = [C18].Value
>             
>             'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>                 "", "RH:In CH:Fd Transpose:Y"
>             'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>             
>             'Ignore cache; get data directly from the Snapshot Server
>             '.Subscribe False
>             'Or use cache by default
>             .Subscribe
>         End With
>         Exit Sub
>         
>     errHandler:
>         MsgBox MyDex2Mgr.GetErrorString(Err.Number)
>     End Sub


#### Python

Things are much simpler in Python:

In [49]:
DEX2TrDf = rd.get_data(
    ['BARC.L', 'TRI.N', 'GB047986974='],  # PSKc1, '92857WAZ3=1M'
    fields=[
        'TR.IR.RatingSourceDescription', 'TR.IR.RatingSourceType', 'TR.IR.Rating',
        'TR.TRBCEconomicSector', 'TR.TRBCEconSectorCode', 'TR.TRBCBusinessSector',
        'TR.TRBCBusinessSectorCode', 'TR.TRBCIndustryGroup', 'TR.TRBCIndustryGroupCode',
        'TR.TRBCIndustry', 'TR.TRBCIndustryCode', 'TR.IR.RatingSourceDescription.date'],
    parameters={'SDate': 0, 'EDate': -3, 'FRQ': 'FY'})  # This will look for a data-point every Fianancial Year for the past 3 Financial Years

In [50]:
# The below in this cell is needed to forward fill our dataframe correctly:
DEX2TrDf.replace({'': np.nan}, inplace=True)
DEX2TrDf.where(pd.notnull(DEX2TrDf), np.nan, inplace=True)

for i in DEX2TrDf.groupby(by=["Instrument"]):
    if i[0] == DEX2TrDf["Instrument"][0]: _DEX2TrDf = i[1].ffill()
    else: _DEX2TrDf = _DEX2TrDf.append(i[1].ffill())
_DEX2TrDf

,Instrument,Rating Source Description,Rating Source Type,Issuer Rating,TRBC Economic Sector Name,TRBC Economic Sector Code,TRBC Business Sector Name,TRBC Business Sector Code,TRBC Industry Group Name,TRBC Industry Group Code,TRBC Industry Name,TRBC Industry Code,Date
0,BARC.L,R&I Long-term Issuer Rating,RII,A,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010,2021-11-26
1,BARC.L,R&I Long-term Issuer Rating,RII,A-,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010,2020-11-18
2,BARC.L,R&I Long-term Issuer Rating,RII,A-,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010,2019-11-27
3,BARC.L,R&I Long-term Issuer Rating,RII,A-,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010,2019-11-27
8,GB047986974=,Fitch Commercial Paper,FCP,F2,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020,2021-07-29
9,GB047986974=,Fitch Commercial Paper,FCP,F2,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020,2021-07-29
10,GB047986974=,Egan-Jones Senior Unsecured,EJU,BBB-,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020,2019-11-26
11,GB047986974=,Egan-Jones Senior Unsecured,EJU,BBB-,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020,2019-11-26
4,TRI.N,NaN,NaN,NaN,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070,NaT
5,TRI.N,Moody's Senior Unsecured,MSU,Baa2,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070,2020-05-11


### DEX2 Fundamental and Reference

<img src="imgs/Dex2 RData Fundamental Data.PNG">

Time Series data as the name suggests changes with time. Reference data, on the other hand, does not primarily change with time; when it does, it usually accounts for a structural change in the instrument at play (e.g.: a company changes from banking to tech operation industries or is acquired and changes name etc). Fundamental data is released temporally say quarterly or half-yearly or annually and is usually looked at across time eg Revenue or Sales. DEX2 provides access to all of our TR fields - which allow you to specify begin dates and end dates amongst other parameters. Access to these is unchanged in our new APIs - only its a lot simpler.

#### VBA

In VBA, after creating '`Private MyDex2Mgr As Dex2Lib.Dex2Mgr`':

>     Private MyDex2Mgr As Dex2Lib.Dex2Mgr
>     Private MyDex2MgrADC As Dex2Lib.IDex2Mgr2
>     Private MyDex2Cookie As Long
>     ' Private variable that holds the Dex2 RData, able to obtain TR fields.
>     Private WithEvents MyDex2RData As Dex2Lib.RData

to complete the Tutorial 6 example, one could go through the following to collect DEX2 Fundamental and Reference data:

>         ' Now create the Dex2 and IDex2 objects.
>         Set MyDex2Mgr = CreateDex2Mgr()
>         Set MyDex2MgrADC = MyDex2Mgr
>         
>         ' Initialise using the DE_MC_ADC_POWERLINK enumeration
>         MyDex2Cookie = MyDex2MgrADC.Initialize(DE_MC_ADC_POWERLINK)
>         ' And create an RData object.
>         Set MyDex2RData = MyDex2Mgr.CreateRData(MyDex2Cookie)
>         
>         With MyDex2RData
>             .InstrumentIDList = Range("G6").Value ' Or for multiple isntruments "TRI.N;MSFT.O;GOOG.O"
>             .FieldList = strInstrList '  Or single field "TR.CLOSE"
>             .RequestParam = "" ' Or of the form "edate:-20d sdate:-9d"
>             .DisplayParam = "CH:Fd" ' Or of the form "RH:In CH:Fd"
>             
>             '' OR can use .SetParameter
>             '.SetParameter Range("G6").Value, strInstrList, "", "CH:Fd"
>             
>             ' Send the query without using cache
>             .Subscribe False  ' Or use cache by default - myDex2RData.Subscribe
>             
>             ' When the data is returned, the myDex2RData_OnUpdate event and Sub are 'fired'.

#### Python

Fundamental data was covered above under 'RData Fundamentals'; such data can be found in Python with [RD](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) (and `rd.get_data`, `rd.get_history` or `rd.content.fundamental_and_reference.Definition`) or [EDAPI](https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api).

#### Fundamental and Reference Timeseries

##### VBA

DEX2 Fundamental and Reference Timeseries data could be fetched similarly to simple time series data:

Start with 

>     ' Private Variable that holds the instance of the Dex2 manager singleton
>     Private MyDex2Mgr As Dex2Lib.Dex2Mgr
>     Private MyDex2MgrADC As Dex2Lib.IDex2Mgr2
>     
>     ' Private variable that holds the cookie that identifies the Dex2 session
>     Private MyDex2Cookie As Long
>     ' Private variable that holds the Dex2 RData
>     Private WithEvents MyDex2RData As Dex2Lib.RData
>     ' Private variable that holds the Dex2 RDataMgr
>     Private MyDex2RDataMgr As Dex2Lib.RDataMgr
>     
>     ' Private variable that holds the Dex2 logger
>     Private MyDex2Logger As CLogger

Then initialise clicks:

>         ' Instantiate the Dex2 manager
>         Set MyDex2Mgr = CreateDex2Mgr()
>         Set MyDex2MgrADC = MyDex2Mgr
>         ' Instantiate the RSearch logger
>         Set MyDex2Logger = New CLogger
>         
>         ' Initialize Dex2 session
>         MyDex2Cookie = MyDex2MgrADC.Initialize(DE_MC_ADC_POWERLINK, MyDex2Logger)
>             
>         ' We can choose to display error code
>         MyDex2Mgr.SetErrorHandling MyDex2Cookie, DE_EH_ERROR_CODES
>         ' Or display error description
>         ' MyDex2Mgr.SetErrorHandling MyDex2Cookie, DE_EH_STRING
>             
>         ' Create a Dex2 query using the session cookie
>         Set MyDex2RData = MyDex2Mgr.CreateRData(MyDex2Cookie)
>         
>         ' Create a Dex2 query manager using the session cookie
>         Set MyDex2RDataMgr = MyDex2Mgr.CreateRDataMgr(MyDex2Cookie)
   
Then, after some error & event handling:

>         ' Set input values
>                 MyDex2RData.SetParameter _
>                     Range("Dex2Item").Value, _
>                     Range("Dex2Fields").Value, _
>                     Range("Dex2RequestParameters").Value, _
>                     Range("Dex2DisplayParameters").Value
>                     ' Or using the following individual properties
>                     ' MyDex2RData.InstrumentIDList = Range("Dex2Item").Value
>                     ' MyDex2RData.FieldList = Range("Dex2Fields").Value
>                     ' MyDex2RData.requestParam = Range("Dex2RequestParameters").Value
>                     ' MyDex2RData.displayParam = Range("Dex2DisplayParameters").Value
>     
>                 ' Send the query without using cache
>                 MyDex2RData.Subscribe (False)
>                 ' Or use cache by default
>                 ' MyDex2RData.Subscribe

Before creating 'OnUpdate' code.

This is all quite heavy in VBA, while it could hardly be simpler in Python:


##### Python

This was covered above, in the section 'RData Fundamentals' using [RD](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python)'s `rd.get_history` function.

## RSearch

RSearch is a COM API function for instrument level search - designed for building instrument lists conforming to various search criteria. See the example below:


<img src="imgs/RSearch.PNG">

[RSearch is a powerful COM API function](https://my.refinitiv.com/content/mytr/en/policies/training-portal/videoarticle.v11535.html). It was best exemplified in [Tutorial 7 - Instrument Search - RSearch
](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-7-instrument-search-r-search)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/rsearch.zip).

### VBA

Creating the sub `cmdRSearch_Click`:

>         ' Instantiate the RSearch manager
>         Set myRSrchMgr = CreateRSearchMgr()
>         
>         If Not myRSrchMgr Is Nothing Then
>             ' Initialize RSearch session (we do not provide any logger here)
>             ' NOTE - This will error if Eikon for Excel is not logged in.
>             myRSrchCookie = myRSrchMgr.Initialize(RS_CT_EIKON)
>             ' Create a RSearch query using the session cookie
>             Set myRSrchQry = myRSrchMgr.CreateRSearchQuery(myRSrchCookie)
>             
>             If Not myRSrchMgr Is Nothing Then
>                 With myRSrchQry
>                     'Initialize the RSearch query with the criteria to use for the search
>                     .AssetClass = [AssetClass].Value
>                     .SearchCriteria = [SearchCriteria].Value '"EPS:>5 RCSIssuerCountryLeaf:Canada"
>                     .SearchParameters = [SearchParameters].Value '"NBROWS:50 SORT:EPS:A"
>                     
>                     ' Send the query
>                     .Send
>                 End With
>             End If
>         End If

Then you can check the status of the query with '`myRSrchQry_OnUpdate`'.

### Python

[RSearch is a powerful COM API function](https://my.refinitiv.com/content/mytr/en/policies/training-portal/videoarticle.v11535.html), and we're happy to announce that the Python equivalent - Search API - is even more powerful. For the full works on it, please do read Nick Zicone's article '[Building Search into your Application Workflow](https://developers.refinitiv.com/en/article-catalog/article/building-search-into-your-application-workflow)'.


The code below is a simple example of how one may use the Search API in Python. It requests M&A data using the filters specified above and orders the data by the announcement date in descending order. More on how you can use search, including guidance, examples, and tips to determine the possible approaches, from simple discovery through experimentation to more advanced techniques, are presented in [this article](https://developers.refinitiv.com/en/article-catalog/article/building-search-into-your-application-workflow).

In [243]:
from refinitiv.data.content import search

In [244]:
bondsSerarch = search.Definition(
    view=search.Views.BOND_FUT_OPT_QUOTES,  # for info on `SearchViews`, you can use `help(search.Views)`
    # specify filtering properties
    filter="IssueCouponRate ge 2",
    # select only the required fields
    select='CommonName, IssuerCommonName, IssueCouponRate, Currency, IssuerCountryName',
    # then specify number of items to be 10000, which is the max; default value is 100
    top=10000
).get_data().data.df

bondsSerarch.head()

,CommonName,IssuerCommonName,IssueCouponRate,Currency,IssuerCountryName
0,TTN 2.375 11/16/24 '24,Titan Global Finance PLC,2.375,EUR,United Kingdom
1,ACB 2.500 02/05/23,Alpha Bank SA,2.5,EUR,Greece
2,BNP 4.000 10/21/23,Banca Nazionale del Lavoro SpA,4.0,EUR,Italy


In [245]:
equitySerarch = search.Definition(
    view=search.Views.EQUITY_QUOTES,  # for info on `SearchViews`, you can use `help(search.Views)`
    filter="MktCapCompanyUsd ge 1000000000000",
    # select only the required fields
    select='CommonName, PriceCloseUsd, MktCapCompanyUsd',
    # then specify number of items to be 10000, which is the max; default value is 100
    top=100
).get_data().data.df

equitySerarch.head()

,CommonName,PriceCloseUsd,MktCapCompanyUsd
0,APPLE ORD,155.96,2506394481920
1,AMAZON COM ORD,129.48,1319084597834
2,MICROSOFT ORD,258.09,1924807313244
3,ALPHABET CL A ORD,109.45,1434013690000
4,SAUDI ARABIAN OIL ORD,9.820359,2160479041916


In [246]:
MnA = search.Definition(
    view=search.Views.DEALS_MERGERS_AND_ACQUISITIONS,  # for info on `SearchViews`, you can use `help(search.Views)`

    # specify filtering properties
    filter="((AcquirerCompanyName ne 'Creditors' and AcquirerCompanyName ne 'Shareholder') and (TargetCountry eq 'US' or TargetCountry eq 'UK')"
    + "and TransactionValueIncludingNetDebtOfTarget ge 100 and TargetPublicStatus eq 'Public')"
    + "and (TransactionStatus eq 'Completed' or TransactionStatus eq 'Pending' or TransactionStatus eq 'Withdrawn')"
    + "and (FormOfTransactionName xeq 'Merger' or FormOfTransactionName xeq 'Acquisition') and (TransactionAnnouncementDate le 2021-11-15 and TransactionAnnouncementDate ge 2020-09-15)",

    # select only the required fields and order them based on announcement date
    # then specify number of items to be 10000, which is the max; default value is 100
    select='TransactionAnnouncementDate, TargetCompanyName, TargetRIC',
    order_by='TransactionAnnouncementDate desc',
    top=10000
).get_data()
MnA = MnA.data.df

#remove companies which doesn't have RIC
MnA = MnA.dropna(subset = ['TargetRIC']).reset_index(drop = True)

print(f'Number of M&A deals for the specified period is {len(MnA)}')
MnA.head()

Number of M&A deals for the specified period is 323


,TransactionAnnouncementDate,TargetCompanyName,TargetRIC
0,2021-11-15,CyrusOne Inc,[CONE.O^C22]
1,2021-11-15,CoreSite Realty Corp,[COR^L21]
2,2021-11-15,LAACO Ltd,[LAACZ.PK^L21]
3,2021-11-15,Casper Sleep Inc,[CSPR.K^A22]
4,2021-11-08,McAfee Corp,[MCFE.O^C22]


A more complex method may be to create a function that incorporates that search function, outputing an excel file. [This](https://gist.github.com/johnukfr/241c2b360a30f96371f430005c8d7738) is a real-life use case replicating RSearch functionality's in looking for Government and Corporate Debt Instruments; the function is rather large, so we put it online for you to access.

## News

### Streaming News


The Office COM API allowed for streaming real time news. This was rather inconvenient because one would have to (i) record news flowing through on their own environment, (ii) be recording with an open streaming session when the news they're looking for is published, (iii) deal with the memory management involved with such streaming data. The Python code below recreated this, but takes a snapshot at the time the code is run:

(N.B: The Accessing News data with the RD or EDAPI Python libraries will only allow you access to news data up to three months ago. For news data predating that, look into [this article](https://developers.refinitiv.com/en/article-catalog/article/news-sentiments---gleaning-insights).)

In [247]:
# Define functions ro be used later:
def display_data(data, instrument, stream):
    clear_output(wait=True)
    current_time = datetime.datetime.now().time()
    print(current_time, "- Data received for", instrument)
    display(data)

In [248]:
# This will trigger the stream:
stream = rd.open_pricing_stream(
    universe=['N2_UBMS'], # N2_NFCP, NFCP_UBMS
    on_data=display_data)

18:04:36.639703 - Data received for N2_UBMS


,PROD_PERM,DSPLY_NAME,PNAC,PROC_DATE,RECORDTYPE,BCAST_TEXT,REG_ID1,REG_FIELD1,STORY_ID,NAMED_ITEM,TAKE_SEQNO,STORY_TYPE,CROSS_REF,ATTRIBTN,MON_PAGES,PROD_CODE,TOPIC_CODE,CO_IDS,LANG_IND,TAKE_TIME,STORY_TIME,STORY_DATE,AREA_ID,SF_NAME,CF_NAME
N2_UBMS,431,1,None,None,None,Waiting for LBM...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1


You could recreate the streeming news too:

In [249]:
def display_news_headline(streaming_prices, instrument_name, fields):
    clear_output(wait=True)
    print(fields['HEADLINE1'])

In [250]:
stream = rd.content.pricing.Definition(universe=['NFCP_UBMS']).get_stream()
stream.on_update(display_news_headline)

<refinitiv.data.content.pricing.Stream object at 0x290e6388 {name='['NFCP_UBMS']'}>

In [251]:
stream.open()

<OpenState.Opened: 'Opened'>

In [252]:
stream.close()

<OpenState.Closed: 'Closed'>

#### NEW FUNCTIONALITY: Get News headlines

We added this functionality to find the headlines of news which was not a functionality in the old COM API:

In [253]:
newsResponse = rd.content.news.headlines.Definition(
    query="LSEG",
    date_from="20.03.2022",
    date_to="25.03.2022",
    count=3
).get_data()  # Something simmilar can be made with EDAPI with, for e.g.`ek.get_news_headlines(query = 'AAPL.O', count=10)`.
newsResponse.data.df

,versionCreated,text,storyId,sourceCode
2022-03-24 18:27:20.000,2022-03-24T18:27:20.000Z,BRIEF-London Stock Exchange Secondary ABO Book...,urn:newsml:reuters.com:20220324:nFWN2VR122:1,NS:RTRS
2022-03-24 18:09:06.000,2022-03-24T18:09:06.000Z,BRIEF-London Stock Exchange Secondary ABO Book...,urn:newsml:reuters.com:20220324:nFWN2VR17K:1,NS:RTRS
2022-03-24 17:56:19.357,2022-03-24T17:56:21.654Z,BLOCKTRADE: LONDON STOCK EXCHANGE SECONDARY A...,urn:newsml:reuters.com:20220324:nFWN2VR17K:2,NS:RTRS


### Historical News

Thankfully, the `ek` library News API is much more powerful and simple to use. Again, most of it is better shown and explained on [GitHub](https://github.com/Refinitiv-API-Samples/Example.DataLibrary.Python/tree/main/Examples), but the below is a great starting point of how to collect News data (that is 15 months old or older):

In [254]:
story_id = newsResponse.data.df.iat[0,2]
story_id

'urn:newsml:reuters.com:20220324:nFWN2VR122:1'

In [255]:
from IPython.display import HTML
story = ek.get_news_story(story_id)
HTML(story)

In [256]:
rd.close_session()

## Conclusion

In conclusion, we can see that the [Office COM API](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office) had many great uses, but limitations too. This was without mentioning its reliability on [DLL](https://docs.microsoft.com/en-us/troubleshoot/windows-client/deployment/dynamic-link-library)s that can be heavy to run on a personal machine. But the Refinitiv Python Libraries ([RD](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python), [RDP](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis) and [EDAPI](https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api)) can not only replicate these COM functionalities but enhance them in many instances, the simplest example being the Historical News functionality shown above.

Several COM API functionalities relying on a technology called Adfin was not replicated in Python in this article, but we will investigate them in another article - so stay tuned!